In [1]:
import pandas as pd
import os
import geopandas as gpd
import folium
import json
import time
import numpy as np
import requests
import osmnx

## Select any of the following metro cities

In [2]:
cities=['Hyderabad','Delhi','Chennai' ,'Mumbai','Pune','Bangalore','Kolkata' ]
print("Choose a city among the following :")
for city in cities:
    print (city)
city = input().title()
if city not in cities:
    print("Wrong city")

Choose a city among the following :
Hyderabad
Delhi
Chennai
Mumbai
Pune
Bangalore
Kolkata


 pune


In [3]:

avg_speed = {'Hyderabad':24.8,
            'Delhi': 23.5,
            'Chennai' :  23.1,
            'Mumbai': 22.7 ,
            'Pune': 22.6,
            'Bangalore' :21.2 ,
            'Kolkata': 19.4 }
city_coord ={
            'Hyderabad':[17.3850, 78.4867],
            'Delhi':  [28.7041, 77.1025],
            'Chennai' : [13.0827, 80.2707],
            'Mumbai':  [19.0760, 72.8777],
            'Pune':  [18.5204, 73.8567],
            'Bangalore' :[12.9716, 77.5946] ,
            'Kolkata': [22.5726, 88.3639] }


average driving speed data souce : Ola Hyperdrive 2019

## Getting shapefile



Reading the city shapefile from the folder 'Data'

In [4]:
repo = "Data"
file = "{}.geojson".format(city)
path_file = os.path.join(repo,file)
city_shapefile= gpd.read_file(path_file).iloc[0]['geometry']

### Functions

In [5]:
def get_amenity(city_shapefile, amenity):
    tags = {"amenity":amenity}
    df = osmnx.geometries.geometries_from_polygon(city_shapefile, tags)
    df.loc[df['geometry'].type=="Polygon", "geometry"] = df['geometry'].to_crs(32644).centroid.to_crs(4326)
    df = df[df['geometry'].type=='Point']
    df=df[df['name'].isnull()==False]
    return df


This function returns a GeoDataFrame of all the required amenities in the city boundary (as defined by the city shapefile)

In [6]:
def get_coords(df):
    long = df['geometry'].x
    lat = df['geometry'].y
    coord_df=pd.concat([long,lat],axis=1)
    #coord_df = police_df[[df['geometry'].x,'lat']].copy()
    
    coord_list = coord_df.values.tolist()
    
    n= int(len(coord_list)/5) + (len(coord_list)%5 !=0 )
    splits = np.array_split(coord_list,n)
    outlist=[]
    for i in splits:
        outlist.append(i.tolist())
    return (outlist,n)

This function returns a 2d list of the amenity's coordinates. 
The Inner list contains a maximum of 5 sets of coordinates as the API used next accepts the maximum of 5 sets of coordinates in one API request.
The length of the outer list is the number of API requests required to be made to receive isochrone maps of all the coordinates.

In [7]:
def get_shapefile(coords,n,time,city):
    import time as t
    key1 ='5b3ce3597851110001cf62483f1e787784844910b048b2846fe2a0b8'
    speed= avg_speed[city]
    d = (speed *1000 * time /60)
    API_output = []
    error =False
    j=1
    for r in coords:
        parameters = {"locations":r,
        "range":[0,d],
        "interval":d,
        "range_type":"distance"}

        head = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': key1,
            'Content-Type': 'application/json; charset=utf-8'
        }
        a = requests.post('https://api.openrouteservice.org/v2/isochrones/driving-car',json = parameters, headers = head)
        if(a.status_code!=200):
            print("ERRORRRRRR!!! API requests limit reached\n Try again after 24 hours")
            error = True
            break;
        API_output.append(a)
        print("done....{}/{}".format(j,n))
        j+=1
        t.sleep(3)
        
    
    #creating shapefile
    if(error==False):
        maps_input = []
        for a in API_output:
            maps_input.append(a.json())
        polygonlist= []
        for in_list in maps_input:
            sdf = gpd.read_file(json.dumps(in_list))
            for i in sdf['geometry']:
                polygonlist.append(i)
        from shapely.ops import unary_union
        cu = unary_union(polygonlist)
        return cu

    return


This functions calls API requests one by one and finds the union of all the returned isochrone maps


## DATA

### POLICE


In [9]:
max_time= int(input("Enter maximim travelling time from nearest police station in mins:"))
df = get_amenity(city_shapefile, "police")
coords,n = get_coords(df)
police = get_shapefile(coords,n,max_time,city)

Enter maximim travelling time from nearest police station in mins: 5


ERRORRRRRR!!! API requests limit reached
 Try again after 24 hours


In [9]:
map_police = folium.Map(location =city_coord[city] , zoom_start=11)
folium.features.GeoJson(police).add_to(map_police)
for i in range(len(df)):
    folium.Marker(location=[df.iloc[i]['geometry'].y,df.iloc[i]['geometry'].x],popup=df.iloc[i]['name']).add_to(map_police)


####  Map highlighting places with a safe travelling time from police staions.

In [10]:
map_police

### Fire station


In [11]:
max_time= int(input("Enter maximim travelling time from nearest fire station in mins:"))
df = get_amenity(city_shapefile, "fire_station")
coords,n = get_coords(df)
fire = get_shapefile(coords,n,max_time,city)

Enter maximim travelling time from nearest fire station in mins: 15


done....1/3
done....2/3
done....3/3


In [12]:
map_fire = folium.Map(location =city_coord[city] , zoom_start=11)
folium.features.GeoJson(fire).add_to(map_fire)
for i in range(len(df)):
    folium.Marker(location=[df.iloc[i]['geometry'].y,df.iloc[i]['geometry'].x],popup=df.iloc[i]['name']).add_to(map_fire)

####  Map highlighting places with a safe travelling time from fire staions.

In [13]:
map_fire

### Hospital



In [24]:
max_time= int(input("Enter maximim travelling time from nearest hospital in mins:"))
df = get_amenity(city_shapefile, "hospital")
coords,n = get_coords(df)
hospital = get_shapefile(coords,n,max_time,city)



sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x196314f38e0>


KeyboardInterrupt: Interrupted by user

In [23]:
map_hospital = folium.Map(location =city_coord[city] , zoom_start=11)
folium.features.GeoJson(hospital).add_to(map_hospital)
for i in range(len(df)):
    folium.Marker(location=[df.iloc[i]['geometry'].y,df.iloc[i]['geometry'].x],popup=df.iloc[i]['name']).add_to(map_hospital)

ValueError: Cannot render objects with any missing geometries: None

####  Map highlighting places with a safe travelling time from hospitals.

In [22]:
map_hospital

NameError: name 'map_hospital' is not defined

## Safest Zones

Places with a safe travelling time from all 3 emergency services

In [87]:
map_final = folium.Map(location =city_coord[city] , zoom_start=11)
mid = police.intersection(fire)
final=mid.intersection(hospital)
folium.features.GeoJson(final).add_to(map_final)

####  Map highlighting places with a safe travelling time from all 3 emergency servies.

In [97]:
map_final